In [1]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain.tools.retriever import create_retriever_tool

In [2]:
model_kwargs = {
    "trust_remote_code": True,
    # "device": "cpu"
    }
encode_kwargs={"normalize_embeddings": True}
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

In [4]:
import pandas as pd
financing_df = pd.read_csv("../data/financing-a-stevens-education.csv")
graduate_tution_df = pd.read_csv("../data/tuition-fees-and-other-expenses-for-graduate-students.csv")
undergraduate_tution_df = pd.read_csv("../data/tuition-fees-and-other-expenses-for-undergraduate-students.csv")
student_life_df = pd.read_csv("../data/student-life.csv")
student_services_df = pd.read_csv("../data/student-services.csv")
graduate_education_df = pd.read_csv("../data/graduate-education.csv")
undergraduate_education_df = pd.read_csv("../data/undergraduate-education.csv")

In [5]:
docs = [
    Document(page_content=text)
    for text in financing_df["text"].tolist()
]
vector = FAISS.from_documents(docs, embeddings)
finance_retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 5})
finance_retriever_tool = create_retriever_tool(
    finance_retriever,
    "financing_a_stevens_education",
    "This contains all the information in regards to helping students finance their education at Stevens by providing information regarding financing sources, institution financing programs, financial aid and state financing",
)

In [5]:
docs = [
    Document(page_content=text)
    for text in undergraduate_tution_df["text"].tolist()
]
vector = FAISS.from_documents(docs, embeddings)
undergraduate_tution_retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 5})
undergraduate_tution_tool = create_retriever_tool(
    undergraduate_tution_retriever,
    "tuition_fees_and_other_expenses_for_undergraduate_students",
    "This contains all the information regarding tution fees and additional fees for undergraduate students",
)

In [6]:
docs = [
    Document(page_content=text)
    for text in graduate_tution_df["text"].tolist()
]
vector = FAISS.from_documents(docs, embeddings)
graduate_tution_retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 5})
graduate_tution_tool = create_retriever_tool(
    graduate_tution_retriever,
    "tuition_fees_and_other_expenses_for_graduate_students",
    "This contains all the information regarding tution fees and additional fees for graduate students",
)

In [7]:
docs = [
    Document(page_content=text)
    for text in student_life_df["text"].tolist()
]
vector = FAISS.from_documents(docs, embeddings)
student_life_retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 5})
student_life_tool = create_retriever_tool(
    student_life_retriever,
    "student_life_at_stevens",
   "This contains all the information regarding student services including sports, code of conduct and student government bodies"
)

In [8]:
docs = [
    Document(page_content=text)
    for text in student_services_df["text"].tolist()
]
vector = FAISS.from_documents(docs, embeddings)
student_services_retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 5})
student_services_tool = create_retriever_tool(
    student_services_retriever,
    "student_services_at_stevens",
    "This contains all the information regarding academic tutoring, career guidance, counseling, disability accommodations, financial aid management, health services, and housing options to facilitate student success and well-being.",
)

In [9]:
docs = [
    Document(page_content=text)
    for text in graduate_education_df["text"].tolist()
]
vector = FAISS.from_documents(docs, embeddings)
graduate_education_retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 5})
graduate_education_tool = create_retriever_tool(
    graduate_education_retriever,
    "graduate_education_at_stevens",
    "This contains all the information regarding graduate student structured academic programs, admissions guidance, course options, grading policies, English proficiency support for international students, and procedures for academic standing and progression.",
)

In [10]:
docs = [
    Document(page_content=text)
    for text in undergraduate_education_df["text"].tolist()
]
vector = FAISS.from_documents(docs, embeddings)
undergraduate_education_retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 5})
undergraduate_education_tool = create_retriever_tool(
    undergraduate_education_retriever,
    "undergraduate_education_at_stevens",
    "This contains all the information regarding undergraduate student structured academic programs, admissions guidance, course options, grading policies, English proficiency support for international students, and procedures for academic standing and progression.",
)

In [11]:
tools = [finance_retriever_tool, undergraduate_tution_tool, graduate_tution_tool, student_life_tool, student_services_tool, graduate_education_tool, undergraduate_education_tool]

In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [17]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [   
        ("system", "You are an assistant for question-answering tasks related to Stevens Institute Of Technology."),
        ("human", """ 
        Use the following pieces of retrieved context to answer the question.
        If you don't know the answer, just say that you don't know.
        If the topic is related to a course then ensure to mention to course numbers and display the result as a table.
        Answer in markdown format and render tables without code 
        Question: {question}
        Context: {agent_scratchpad}
        Answer:"""),
    ]
)
print(chat_template)

input_variables=['agent_scratchpad', 'question'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an assistant for question-answering tasks related to Stevens Institute Of Technology.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'question'], template=" \n        Use the following pieces of retrieved context to answer the question.\n        If you don't know the answer, just say that you don't know.\n        If the topic is related to a course then ensure to mention to course numbers and display the result as a table.\n        Answer in markdown format and render tables without code \n        Question: {question}\n        Context: {agent_scratchpad}\n        Answer:"))]


In [18]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, chat_template)

In [19]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [20]:
agent_executor.invoke({"question": "Give me information regarding financial aid at Stevens"})



> Entering new AgentExecutor chain...

Invoking: `financing_a_stevens_education` with `{'query': 'financial aid'}`




	Financial Aid Sources
Many sources of financial aid are available to graduate students. These include fellowships, assistantships, on-campus employment, employer tuition assistance plans, loan funds, and deferred payment plans. Fellowships and assistantships are granted on a competitive basis to outstanding full-time graduate students. Applicants should consult their department for more information regarding assistantships. Continuing students may become eligible for additional sources of aid as they progress through the program and should consult with their department at regular intervals.
Click here for more information.

Graduate AssistantshipsProvost Masters Fellowship AwardsResearch AssistantshipsTeaching AssistantshipsRobert Crooks Stanley Graduate FellowshipsGraduate GrantLoans and Work-StudyFAFSAFederal LoansPrivate Education Loans





	Federal Financial Ai

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 16504 tokens (16085 in the messages, 419 in the functions). Please reduce the length of the messages or functions.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}